In [3]:
import numpy as np
import MDAnalysis as mda
from MDAnalysis.tests.datafiles import PSF, DCD, GRO, XTC
np.set_printoptions(precision=6, suppress=True)

In [153]:
x_0 = np.array([
    [2], 
    [3], 
    [1], 
    [2], 
    [5], 
])

M = np.array([
    [1,1], 
    [1,0], 
    [0,1], 
    [0,1], 
    [0,1], 
])

M_ = np.array([
    [1 , 0, 0, 0], 
    [-1, 0, 0, 0], 
    [-1, 0, 1, 1], 
    [0 , 1,-1,-1], 
    [0 ,-1, 0, 0], 
])


S = np.array([
    [1,0 ], 
    [1,0 ], 
    [1 ,0 ], 
    [0 ,1 ], 
    [0 ,1 ], 
])


D = np.array([
    [0 ,1 ], 
    [1 ,0 ],  
])


m_c = np.array([
    [0.2,], 
    [0.3,],
])

q_c = np.array([
    [-3,], 
    [1,],
])

a = -np.log(np.array([
    [4], 
    [1], 
    [2], 
    [1], 
    [3], 
]))

k = np.array([
    [1], 
    [0.7], 
    [0.1], 
    [0],  
])

v = np.array([
    [0], 
    [0], 
    [0], 
    [0.8],  
])

c = np.array([
    [0.1], 
    [0.1], 
    [0.1], 
    [0.2], 
    [0.1], 
])


x_h = np.array([
    [0], 
    [0], 
    [0], 
    [0], 
    [3], 
])


h = np.array([
    [0], 
    [0], 
    [0], 
    [0], 
    [2], 
])

###

p_0 = np.array([
    [0], 
    [0], 
    [0], 
    [0], 
    [0], 
])

In [154]:
x, p = x_0, p_0
q =  np.dot(  np.diag( np.dot(M, q_c).T[0] ), S )
V = np.dot( q, np.dot(1/(D+1)-1, q.T) )
m = np.dot( M, m_c)

In [173]:
def relu(x) :
    return np.maximum(0,x)

def Energy(x,p) :
    global a, V, m
    
    energy = np.dot(x.T, np.log(x)) + np.dot(a.T, x) + 0.5*np.dot(x.T, np.dot(V, x)) + 0.5*(np.dot( (1/m.T), (p*p*x)))
    
    return energy[0]

def Flow(x,p) :
    global M_, k, v, h, x_h, c
    
    grad_x = (np.log(x)+1).T + a.T + np.dot(x.T, V) + (0.5*(1/m)*p*p).T
    grad_p = (0.5*(1/m)*p*x).T
    
    chemical_flow_x = -np.dot(M_, (k.T*(np.exp( np.dot(grad_x, M_)) -1 )*(np.exp(-0.5*np.dot(grad_x, M_) + 0.5*np.dot(np.log(x.T), np.abs(M_)))) ).T)
    hamiltonian_flow_x = np.dot(M_, v*(np.dot(grad_p, M_).T) )
    hamiltonian_flow_p = -np.dot(M_, v*(np.dot(grad_x, M_).T) )
    colision_flow_p = -c*(grad_p).T
    external_homeostasis_flow_x = h*(x_h-x)
    
    return chemical_flow_x + hamiltonian_flow_x,  hamiltonian_flow_p + colision_flow_p
    #return chemical_flow_x + hamiltonian_flow_x + external_homeostasis_flow_x,  hamiltonian_flow_p + colision_flow_p

In [174]:
flow_x, flow_p = Flow(x,p) 
flow_x, flow_p

(array([[  6.137168],
        [ -6.137168],
        [ 26.554393],
        [-33.095706],
        [  0.404145]]),
 array([[-0.      ],
        [-0.      ],
        [ 8.709035],
        [-8.709035],
        [-0.      ]]))

In [170]:
energy = Energy(x,p)

In [171]:
np.dot(M.T, flow_x)

array([[ 0.],
       [-4.]])

In [4]:
print(mda.Universe(PSF, DCD))
print(mda.__version__)

<Universe with 3341 atoms>
2.1.0


In [5]:
psf = mda.Universe(PSF)
print(psf)
print(hasattr(psf, 'trajectory'))

<Universe with 3341 atoms>
False


C:\Users\arizona95\anaconda3\envs\tensorflow\lib\site-packages\MDAnalysis\core\universe.py:148: UserWarning: No coordinate reader found for C:\Users\arizona95\anaconda3\envs\tensorflow\lib\site-packages\MDAnalysisTests\data\adk.psf. Skipping this file.
  'this file.'.format(filename))


In [6]:
gro = mda.Universe(GRO)
print(gro)
print(len(gro.trajectory))

<Universe with 47681 atoms>
1


In [8]:
u = mda.Universe(PSF, DCD)
print(u)
print(len(u.trajectory))

<Universe with 3341 atoms>
98


In [9]:
print(u.residues)

<ResidueGroup [<Residue MET, 1>, <Residue ARG, 2>, <Residue ILE, 3>, ..., <Residue ILE, 212>, <Residue LEU, 213>, <Residue GLY, 214>]>


In [10]:
u.atoms

<AtomGroup with 3341 atoms>

In [11]:
last_five = u.atoms[-5:]
print(last_five)

<AtomGroup [<Atom 3337: HA1 of type 6 of resname GLY, resid 214 and segid 4AKE>, <Atom 3338: HA2 of type 6 of resname GLY, resid 214 and segid 4AKE>, <Atom 3339: C of type 32 of resname GLY, resid 214 and segid 4AKE>, <Atom 3340: OT1 of type 72 of resname GLY, resid 214 and segid 4AKE>, <Atom 3341: OT2 of type 72 of resname GLY, resid 214 and segid 4AKE>]>


In [12]:
nhh = u.atoms[:3]
print(nhh.names)

['N' 'HT1' 'HT2']


In [13]:
angle_nhh = nhh.angle
print(angle_nhh.value())

37.99234750892497
